In [67]:
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever

from langchain_core.messages import HumanMessage, SystemMessage

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

from IPython import display

from dotenv import load_dotenv


In [3]:

!pip install weave


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import weave

weave.init("rag-baselining")

There are different credentials in the netrc file and the environment. Using the environment value.


Logged in as Weights & Biases user: pasindusankalpa599.
View Weave data at https://wandb.ai/pasindusankalpa599-pasindu/rag-baselining/weave


In [68]:
load_dotenv()

True

In [6]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [90]:
text_model =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=800,
)

In [8]:
def extract_images_and_tables_from_unstructured_pdf(pdf_path,output_path):
    # Load the PDF file
    raw_pdf_elements=partition_pdf(
    filename=pdf_path,                  
    strategy="hi_res",                                
    extract_images_in_pdf=True,                      
    extract_image_block_types=["Image", "Table"],          
    extract_image_block_to_payload=False,                  
    extract_image_block_output_dir=output_path, 
    )
    
    img=[]
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))

    tab=[]
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

    
    return img,tab


In [9]:

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

In [66]:
def summarize_image(encoded_image, vision_model):
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval. It should be included the most important information in the image.
    The summary should be limited to 100 characters. \
      """

    msg = [
        HumanMessage(content=[
            {
                "type": "text",
                "text": prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ])
    ]
    response = vision_model.invoke(msg)
    return response.content

In [11]:
import uuid
def get_summary_of_images(image_elements,output_path, vision_model):
    image_summaries = []
    img_base64_list = []
    img_path_list = []
    for i in os.listdir(output_path):
        if i.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(output_path, i)
            encoded_image = encode_image(image_path)
            img_base64_list.append(encoded_image)
            summary = summarize_image(encoded_image, vision_model)
            print("summary", summary)
            image_summaries.append(summary)

        image_path = os.path.join(output_path, i)
        print("rename image path", image_path)
        os.rename(image_path, os.path.join("/images",uuid.uuid4().hex + i))
        #add unique id to image path
        image_path = os.path.join("/images", uuid.uuid4().hex + i)
        
        

        img_path_list.append(image_path)




    print(image_summaries , "image summaries")
    print(img_base64_list , "image base64 list")
    print(img_path_list , "image path")
    return image_summaries,img_base64_list, img_path_list

In [12]:
import uuid
print (uuid.uuid4())

43ac532e-021b-46f3-9eb8-44d7d3636dd9


In [13]:
def get_summary_of_tables(table_elements,text_model):
    table_summaries = []

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table:{element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    summarize_chain = {"element": lambda x: x} | prompt | text_model | StrOutputParser()

    table_summaries = summarize_chain.batch(table_elements)

    return table_summaries

In [14]:
#create documents for tables summary
def create_documents_tables(table_elements,table_summaries,documents,retrieve_contents):
    i = str(uuid.uuid4())
    for e, s in zip(table_elements, table_summaries):
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'table',
                'original_content': e
            }
        )
        retrieve_contents.append((i, e))
        documents.append(doc)

In [15]:
def get_summary_of_images(image_elements,output_path, vision_model):
    image_summaries = []
    img_base64_list = []
    img_path_list = []
    for i in os.listdir(output_path):
        if i.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(output_path, i)
            encoded_image = encode_image(image_path)
            img_base64_list.append(encoded_image)
            summary = summarize_image(encoded_image, vision_model)
            image_summaries.append(summary)

        image_path = os.path.join(output_path, i)
        print("rename image path", image_path)

        destination_dir = "./images-processed/"
        os.makedirs(destination_dir, exist_ok=True)
        unique_filename = uuid.uuid4().hex + i
        new_image_path = os.path.join(destination_dir, unique_filename)
        os.rename(image_path, new_image_path)
        #add unique id to image path
        img_path_list.append(new_image_path)

    print(image_summaries , "image summaries")
    # print(img_base64_list , "image base64 list")
    # print(img_path_list , "image path")
    return image_summaries,img_base64_list , img_path_list

In [30]:
def create_documents_images(img_base64_list,image_summaries,documents,retrieve_contents, img_path_list):
    i = str(uuid.uuid4())
    for e, s , p in zip(img_base64_list, image_summaries , img_path_list):
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'image',
                'original_content': p,
                'base64': e
            }
        )
        retrieve_contents.append((i, p))
        documents.append(doc)

In [16]:
def get_documents_text(path_name):
        
    loader = PyPDFLoader(path_name)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    return docs


In [17]:
def create_documents_text(text_documents,documents):
    i = str(uuid.uuid4())
    for s in text_documents:
        doc = Document(
            page_content = s.page_content,
            metadata = {
                'id': i,
                'type': 'text',
                'original_content': s.metadata['source']
            }
        )
        # print(doc)
        documents.append(doc)


In [18]:

#create vector store
def create_vector_store(documents, embedding_model, dbpath):
    vectorstore = Chroma.from_documents(documents=documents, embedding=embedding_model , persist_directory=dbpath)
    return vectorstore


In [19]:
#create load vector store
def load_vector_store(directory, embedding_model):
    vectorstore = Chroma(
        embedding_function=embedding_model,
        persist_directory=directory
    )
    return vectorstore


In [20]:

#create chain
@weave.op()
def create_chain(text_model):
    prompt_template = """
    You are a tutor assistant. You aims to provide personalized instruction, guided problem-solving, and adaptive teaching to cater to each student's unique needs and learning pace.
    Answer the question based only on the following context, which can include text, images and tables:
    {context}
    Question: {question}
    Don't answer if you have no context and decline to answer and say "Sorry, I don't have much information about it."
    Just return the helpful answer in as much as detailed possible.
    Answer:
    """
    qa_chain = LLMChain(llm=text_model,
                        prompt=PromptTemplate.from_template(prompt_template))
    
    return qa_chain

In [128]:
@weave.op()
def retrieve_content(query,chain,vectorstore):
    relevant_docs = vectorstore.similarity_search(query , k = 2)
    print(relevant_docs)
    context = ""
    relevant_images = []
    for d in relevant_docs:
        if d.metadata['type'] == 'text':
            context += '[text]' + d.page_content
        elif d.metadata['type'] == 'table':
            context += '[table]' + d.page_content
        elif d.metadata['type'] == 'image':
            context += '[image]' + d.page_content
            relevant_images.append(d.metadata['original_content'])
    result = chain.run({'context': context, 'question': query})
    print(relevant_images , "relevant images")
    return result, relevant_images , context

In [129]:
class RAGModel(weave.Model):
    model_name: str = "text-embedding-3-small"
        
    @weave.op()
    def predict(self, question: str) -> dict:
        # Perform the actual retrieval within the method
        vectorstore = load_vector_store("doc1/v1", OpenAIEmbeddings(model=self.model_name))
        chain = create_chain(text_model)

        result, relevant_images , context = retrieve_content(question,chain, vectorstore)
        return {"answer": result, "context": context}


In [97]:
pathname1 = "pdf1.pdf"
pathname2 = "What_is_a_Battery.pdf"
text_model = text_model
vision_model = text_model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
fpath="./extracted_data/"
database_path = 'docs/chroma_db/'

In [98]:
#  call image and table pdf documents
image_elements, table_elements = extract_images_and_tables_from_unstructured_pdf(pathname1,fpath)

KeyboardInterrupt: 

In [ ]:
#  call image and table pdf documents
image_elements, table_elements = extract_images_and_tables_from_unstructured_pdf(pathname1,fpath)

In [ ]:
table_elements

['Learn to code solving problems with our hands-on Python course! Try Programiz PRO today. (hps://programiz.pro/learn/master-python? utm_source=sticky- banner&utm_campaign=programiz&utm_medium=referral) Programiz PRO (hps://programiz.pro/learn/master-python? utm_source=nav- (/) floating&utm_campaign=programiz&utm_medium=referral)',
 'Programiz PRO',
 '** to get a to the power b Learn to code solving problems with our hands-on Python course! Try Programiz PRO today. Programiz PRO (hps://programiz.pro/learn/master-python? utm_source=nav- (/) floating&utm_campaign=programiz&utm_medium=referral) www.domain-name.com',
 'Operator Name Example = Assignment Operator a = 7 += Addition Assignment a += 1 # a = a + 1 -= Subtraction Assignment a -= 3 # a = a - 3 *= Multiplication Assignment a *= 4 # a = a * 4 /= Division Assignment a /= 3 # a = a / 3',
 'Learn to code solving problems with our hands-on Python course! Try Programiz PRO today. a **= 10 # a = a ** 10 Exponent Assignment **= (hps://pro

In [ ]:
table_summaries = get_summary_of_tables(table_elements,text_model)

In [ ]:
table_summaries

['Summary: Programiz offers a hands-on Python course designed to teach coding through problem-solving. Explore Programiz PRO for more details and enrollment options.',
 '**Table Summary: Programiz PRO Features and Benefits**  \nThis table outlines the key features and benefits of the Programiz PRO subscription, including access to premium content, interactive coding exercises, personalized learning paths, and additional resources for programming languages. It highlights pricing tiers, user testimonials, and comparisons with free offerings, emphasizing the enhanced learning experience for users.',
 '**Table Summary:** This table promotes a hands-on Python coding course offered by Programiz PRO. It includes a link to the course for further details and emphasizes practical problem-solving skills.',
 '**Summary:** This table lists assignment operators in programming, including their names and examples. Key operators include:  \n- `=` (Assignment): `a = 7`  \n- `+=` (Addition Assignment): `

In [ ]:
image_summaries, images_list , image_path_list = get_summary_of_images(image_elements,fpath,vision_model)

rename image path ./extracted_data/figure-1-1.jpg
rename image path ./extracted_data/figure-1-2.jpg
rename image path ./extracted_data/figure-1-3.jpg
rename image path ./extracted_data/figure-10-17.jpg
rename image path ./extracted_data/figure-11-18.jpg
rename image path ./extracted_data/figure-11-19.jpg
rename image path ./extracted_data/figure-12-20.jpg
rename image path ./extracted_data/figure-12-21.jpg
rename image path ./extracted_data/figure-12-22.jpg
rename image path ./extracted_data/figure-12-23.jpg
rename image path ./extracted_data/figure-12-24.jpg
rename image path ./extracted_data/figure-13-25.jpg
rename image path ./extracted_data/figure-2-4.jpg
rename image path ./extracted_data/figure-2-5.jpg
rename image path ./extracted_data/figure-3-6.jpg
rename image path ./extracted_data/figure-3-7.jpg
rename image path ./extracted_data/figure-4-8.jpg
rename image path ./extracted_data/figure-4-9.jpg
rename image path ./extracted_data/figure-5-10.jpg
rename image path ./extracted_d

In [ ]:
documents = []
retrieve_contents = []
create_documents_images(images_list,image_summaries,documents,retrieve_contents,image_path_list)

In [ ]:
embedding_model =  OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = create_vector_store(documents, embedding_model, "doc1/v1")

In [ ]:
image_summaries

['Image summary: A table displaying two logical operators, "in" and "not in," with their meanings and examples. The "in" operator indicates that a value/variable is found in a sequence, while the "not in" operator indicates that a value/variable is not found in the sequence. Examples provided include "5 in x" and "5 not in x."',
 'Image features a webpage promoting a hands-on Python course called "Programiz PRO." It includes links to the course and a "Check Code" button. The page emphasizes coding problem-solving skills and offers multiple URLs for course access.',
 'Image summary: Title "Operators in Python #5" with a prominent YouTube play button, indicating video content related to Python programming operators.',
 'Image summary: The image displays a webpage promoting a Python programming course from Programiz, featuring links to share the article on Facebook and Twitter. It includes a feedback section with smiley face icons for users to indicate if they found the article helpful.',

In [ ]:
image_summaries, images_list  = get_summary_of_images(image_elements,fpath,vision_model)

rename image path ./extracted_data/figure-1-1.jpg
rename image path ./extracted_data/figure-1-2.jpg
rename image path ./extracted_data/figure-1-3.jpg
rename image path ./extracted_data/figure-1-4.jpg
rename image path ./extracted_data/figure-2-5.jpg
rename image path ./extracted_data/figure-2-6.jpg
rename image path ./extracted_data/figure-2-7.jpg
rename image path ./extracted_data/figure-3-8.jpg
rename image path ./extracted_data/figure-3-9.jpg
['Logo of SparkFun, featuring the brand name "sparkfun" in a bold font with the tagline "START SOMETHING" underneath. The logo includes a stylized flame or wrench icon above the text.', 'A person in a suit holds a large, vibrant red heart, symbolizing love or affection, against a plain white background.', 'Image summary: A text-based graphic titled "What is a Battery?" featuring a central YouTube play button icon, indicating a video resource related to the topic of batteries.', 'Image of a lithium polymer battery connected to a USB charging cir

In [ ]:
text_documents = get_documents_text(pathname1)

In [ ]:
text_documents

[Document(metadata={'source': 'Introduction to_Electric_Motors.pdf', 'page': 0}, page_content='Introduction to Electric Motors\nHi Guys! W elcome you onboard. Interested to know the Introduction to  Electric Motors ?Keep reading…A   motor  is an electrical\nmachine...\nPosted at: 08 - Sep - 2020 Category: Tutorials Author: jameswilson 0 Comments\nHi Guys! W elcome you onboard. Interested to know the Introduction to Electric Motors? Keep\nreading… A motor is an electrical machine that converts electrical energy into mechanical\nenergy . It works exactly opposite to generator that converts mechanical energy to electrical\nenergy . The first electric motors were introduced by Andrew Gordon and Benjamin Franklin in\ntheir experiment in 1740 which were nothing but electrostatic devices. From household to\nindustrial applications, you’ll see motors everywhere. Motors can be divided into two main\ncategories:\nMotors in cars, rectifiers, and batteries are a source of direct current motors whi

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks



In [ ]:

pdf_docs = [pathname1]
text = get_pdf_text(pdf_docs)

In [ ]:
text

"Introduction to Electric Motors\nHi Guys! W elcome you onboard. Interested to know the Introduction to  Electric Motors ?Keep reading…A   motor  is an electrical\nmachine...\nPosted at: 08 - Sep - 2020 Category: Tutorials Author: jameswilson 0 Comments\nHi Guys! W elcome you onboard. Interested to know the Introduction to Electric Motors? Keep\nreading… A motor is an electrical machine that converts electrical energy into mechanical\nenergy . It works exactly opposite to generator that converts mechanical energy to electrical\nenergy . The first electric motors were introduced by Andrew Gordon and Benjamin Franklin in\ntheir experiment in 1740 which were nothing but electrostatic devices. From household to\nindustrial applications, you’ll see motors everywhere. Motors can be divided into two main\ncategories:\nMotors in cars, rectifiers, and batteries are a source of direct current motors while motors incorporated in electrical generators, power grid\nstations, and invertors operate b

In [ ]:
list  = get_text_chunks(text)

In [ ]:
len(list)

22

In [ ]:
list[0]

'Introduction to Electric Motors\nHi Guys! W elcome you onboard. Interested to know the Introduction to  Electric Motors ?Keep reading…A   motor  is an electrical\nmachine...\nPosted at: 08 - Sep - 2020 Category: Tutorials Author: jameswilson 0 Comments\nHi Guys! W elcome you onboard. Interested to know the Introduction to Electric Motors? Keep\nreading… A motor is an electrical machine that converts electrical energy into mechanical\nenergy . It works exactly opposite to generator that converts mechanical energy to electrical\nenergy . The first electric motors were introduced by Andrew Gordon and Benjamin Franklin in\ntheir experiment in 1740 which were nothing but electrostatic devices. From household to\nindustrial applications, you’ll see motors everywhere. Motors can be divided into two main\ncategories:\nMotors in cars, rectifiers, and batteries are a source of direct current motors while motors incorporated in electrical generators, power grid'

In [ ]:
def create_documents_text(text_list ,documents):
    i = str(uuid.uuid4())
    for s in text_list:
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'text',
        
            }
        )
        # print(doc)
        documents.append(doc)

In [ ]:
documents = []

In [ ]:
create_documents_text(list,documents)

In [ ]:
documents

[Document(metadata={'id': '4a03f07f-9907-4bbb-87bb-23955cabeb26', 'type': 'text'}, page_content="ELECTRONICS - ITask 2 : Traffic Lights\nYou can set the Green LED\nTake a look back at 3 configurations for PUSH BUTTONSA Battery is like a water tank!\nWater Amount = Charge\nPressure at hose end = Voltage\nWater Flow = Current\nMore Water in the tank means more pressure at the end of the hose! \nMore Charge in the battery means more Voltage between the two terminalsWhat will happen when\nwater level gets decreased?\nTurbine will rotate slowerTorch light will dimWhat will happen when\ntorch battery gets drained?\nWe measure Current flow in Amps (A)Voltage and Current\nTank 1 Tank 2 Tank 1 Tank 2\nIncrease Tank 2 \nWater Amount\nWhen Voltage  Increases , Current IncreasesWhat about the pipe width?\nLess Resistance\nHigh Resistance \nWhen voltage is same, High Resistance\nLow ResistanceLow Current\nHigh CurrentBeautiful Relationship of Voltage, Current and Resistance\nWe call it Ohm's LawRes

In [ ]:
embedding_model =  OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = create_vector_store(documents, embedding_model, "doc1/v1")

In [ ]:
import streamlit as st

# File uploader for PDFs
uploaded_file = st.file_uploader("Upload a PDF", type=["pdf"])

if uploaded_file is not None:
    # Save the uploaded file locally
    with open(f"uploaded_{uploaded_file.name}", "wb") as f:
        f.write(uploaded_file.read())
    
    st.success(f"File {uploaded_file.name} has been saved successfully as uploaded_{uploaded_file.name}")


2024-09-21 10:35:27.640 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 10:35:27.641 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 10:35:27.648 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 10:35:27.871 
  command:

    streamlit run d:\Documnets\ML\LLM\New-multimodel-pdf-llm\env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-21 10:35:27.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
model = RAGModel()

In [ ]:
model.predict("What is Ohms law")

[Document(metadata={'id': '4a03f07f-9907-4bbb-87bb-23955cabeb26', 'type': 'text'}, page_content="Low ResistanceLow Current\nHigh CurrentBeautiful Relationship of Voltage, Current and Resistance\nWe call it Ohm's LawResistor Color Codes\nBrown Black Red Gold\n1 0 x100 5%\n1000 Ohms10 x 100 Let ’s find the Values of Resistors In your box!Let ’s recap these Symbols\nBattery\nResistorPush button\nSwitch\nLEDLet ’s draw a schematic!\nh t t p s : / / t i n y u r l . c o m / 2 c b t d k m t"), Document(metadata={'base64': '...', 'id': '97512b6a-0a19-4aab-81df-6ed0bd57c6e8', 'original_content': './images-processed/378e6ab4433d4b6fa64c630e2810afa1figure-8-15.jpg', 'type': 'image'}, page_content='Characters representing Voltage, Amperage, and Ohm in a humorous electrical concept illustration.'), Document(metadata={'id': '4a03f07f-9907-4bbb-87bb-23955cabeb26', 'type': 'text'}, page_content="ELECTRONICS - ITask 2 : Traffic Lights\nYou can set the Green LED\nTake a look back at 3 configurations for

{'answer': "Ohm's Law is a fundamental principle in electronics that describes the relationship between voltage (V), current (I), and resistance (R) in an electrical circuit. It can be expressed with the formula:\n\n\\[ V = I \\times R \\]\n\nWhere:\n- \\( V \\) is the voltage measured in volts (V),\n- \\( I \\) is the current measured in amperes (A), and\n- \\( R \\) is the resistance measured in ohms (Ω).\n\nAccording to Ohm's Law:\n- When the voltage increases, the current also increases if the resistance remains constant.\n- Conversely, if the resistance increases while the voltage stays the same, the current will decrease.\n\nThis relationship illustrates that voltage can be thought of as the pressure in a water system, current as the flow of water, and resistance as the width of the pipe. A greater voltage (or pressure) leads to a higher current (or flow), while increased resistance (narrower pipe) results in a lower current. \n\nIn summary, Ohm's Law provides a crucial framework

In [114]:
questions = [
    {"question": "How does Ohm's Law explain the relationship between voltage, current, and resistance?"},
    {"question": "What happens to the brightness of a torch light when the battery is drained, and how is it similar to water flow in a tank system?"},
    {"question": "Can you explain how resistor color codes work and how to calculate the resistance using them?"},
    {"question": "What is the role of resistance in controlling the current flow when the voltage is constant?"},
    {"question": "How can you design a basic circuit using symbols like a battery, resistor, push button, switch, and LED?"},
    {"question": "How do push buttons work in different configurations, and how can they be used to control a traffic light system in a circuit?"}
]


In [120]:
import json

@weave.op()
async def context_precision_score(question, model_output):
    context_precision_prompt = """Given the question, answer, and context, verify whether the context helped in reaching the provided answer.
In the JSON output, if the context was useful, output {{verification: 1}}.
If the context was not useful, output {{verification: 0}}.
Output in valid JSON format only.

Context:
```
{context}
```

Answer:
{answer}
"""

    client = OpenAI()
    prompt = context_precision_prompt.format(
        question=question,
        context=model_output["context"],
        answer=model_output["answer"],
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" }
    )
    response_message = response.choices[0].message
    response = json.loads(response_message.content)
    return {
        "verdict": 1 == 1,
    }

In [99]:
import nest_asyncio
nest_asyncio.apply()

In [130]:
# weave.Evaluation
evaluation = weave.Evaluation(
    dataset=questions,
    scorers=[context_precision_score]
)

In [131]:
import asyncio
model = RAGModel()
asyncio.run(evaluation.evaluate(model))

[Document(metadata={'id': '4a03f07f-9907-4bbb-87bb-23955cabeb26', 'type': 'text'}, page_content="Low ResistanceLow Current\nHigh CurrentBeautiful Relationship of Voltage, Current and Resistance\nWe call it Ohm's LawResistor Color Codes\nBrown Black Red Gold\n1 0 x100 5%\n1000 Ohms10 x 100 Let ’s find the Values of Resistors In your box!Let ’s recap these Symbols\nBattery\nResistorPush button\nSwitch\nLEDLet ’s draw a schematic!\nh t t p s : / / t i n y u r l . c o m / 2 c b t d k m t"), Document(metadata={'base64': '...', 'id': '97512b6a-0a19-4aab-81df-6ed0bd57c6e8', 'original_content': './images-processed/378e6ab4433d4b6fa64c630e2810afa1figure-8-15.jpg', 'type': 'image'}, page_content='Characters representing Voltage, Amperage, and Ohm in a humorous electrical concept illustration.')]
[Document(metadata={'base64': '...', 'id': '97512b6a-0a19-4aab-81df-6ed0bd57c6e8', 'original_content': './images-processed/2145190dd4e642feb7ffa397787d9dacfigure-10-17.jpg', 'type': 'image'}, page_conten

Evaluated 1 of 6 examples

Evaluated 2 of 6 examples

Evaluated 3 of 6 examples

Evaluated 4 of 6 examples

Evaluated 5 of 6 examples

Evaluated 6 of 6 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 6, 'true_fraction': 1.0}},
    'model_latency': {'mean': 10.43888521194458}
}

🍩 https://wandb.ai/pasindusankalpa599-pasindu/rag-baselining/r/call/019282fb-1350-7a10-a64c-e6bdabb4b060


{'context_precision_score': {'verdict': {'true_count': 6,
   'true_fraction': 1.0}},
 'model_latency': {'mean': 10.43888521194458}}